In [138]:
import tensorflow as tf
from read_data import read_data, get_squad_data_filter, update_config
from tensorflow.contrib.rnn.python.ops.rnn_cell import _linear
import flag as fg
import os
import json
import numpy as np

In [2]:
config = fg.main(_)

In [3]:
# variables

batch_size = 100
N = batch_size

In [4]:
config.out_dir = os.path.join(config.out_base_dir, config.model_name, str(config.run_id).zfill(2))


assert config.load or config.mode == 'train', "config.load must be True if not training"
if not config.load and os.path.exists(config.out_dir):
    shutil.rmtree(config.out_dir)

config.save_dir = os.path.join(config.out_dir, "save")
config.log_dir = os.path.join(config.out_dir, "log")
config.eval_dir = os.path.join(config.out_dir, "eval")
config.answer_dir = os.path.join(config.out_dir, "answer")
if not os.path.exists(config.out_dir):
    os.makedirs(config.out_dir)
if not os.path.exists(config.save_dir):
    os.mkdir(config.save_dir)
if not os.path.exists(config.log_dir):
    os.mkdir(config.log_dir)
if not os.path.exists(config.answer_dir):
    os.mkdir(config.answer_dir)
if not os.path.exists(config.eval_dir):
    os.mkdir(config.eval_dir)

In [5]:
data_filter = get_squad_data_filter(config)

train_data = read_data(config, 'train', False, data_filter=data_filter)
dev_data = read_data(config, 'dev', False, data_filter=data_filter)

Loaded 87507/87599 examples from train
Loaded 10544/10570 examples from dev


In [6]:
train_data

In [64]:
update_config(config, [train_data, dev_data])


In [10]:
word2vec_dict = train_data.shared['lower_word2vec'] if config.lower_word else train_data.shared['word2vec']
word2idx_dict = train_data.shared['word2idx']


In [11]:
idx2vec_dict = {word2idx_dict[word]: vec for word, vec in word2vec_dict.items() if word in word2idx_dict}
emb_mat = np.array([idx2vec_dict[idx] if idx in idx2vec_dict
                    else np.random.multivariate_normal(np.zeros(config.word_emb_size), np.eye(config.word_emb_size))
                    for idx in range(config.word_vocab_size)])
config.emb_mat = emb_mat

In [15]:
pprint(config.__flags, indent=2)

Pretty printing has been turned OFF


In [36]:
global_step = tf.get_variable('global_step', shape=[], dtype='int32', initializer=tf.constant_initializer(0), trainable=False)

In [126]:
# Context and Ques Parameters
N = config.batch_size
M = config.max_num_sents
JX = config.max_sent_size
JQ = config.max_ques_size
VW = config.word_vocab_size
VC = config.char_vocab_size
W = config.max_word_size
d =  config.hidden_size
dc = config.char_emb_size
dw = config.word_emb_size
dco = config.char_out_size

print("N:", N)
print("M:", M)
print("JX:", JX)
print("JQ:", JQ)
print("VW:", VW)
print("VC:", VC)
print("W:", W)
print("d:", d)

N: 60
M: 1
JX: 866
JQ: 30
VW: 1224
VC: 281
W: 16
d: 100


In [107]:

X = train_data.shared['x']

In [111]:
import itertools

In [112]:
xi = [list(itertools.chain(*X[0]))]


In [123]:
# Placeholders

x = tf.placeholder('int32', [N, None, None], name='x')
cx = tf.placeholder('int32', [N, None, None, W], name='cx')
x_mask = tf.placeholder('bool', [N, None, None], name='x_mask')
q = tf.placeholder('int32', [N, None], name='q')
cq = tf.placeholder('int32', [N, None, W], name='cq')
q_mask = tf.placeholder('bool', [N, None], name='q_mask')
y = tf.placeholder('bool', [N, None, None], name='y')
y2 = tf.placeholder('bool', [N, None, None], name='y2')
is_train = tf.placeholder('bool', [], name='is_train')

In [127]:
with tf.variable_scope("emb"):
    if config.use_char_emb:
        with tf.variable_scope("emb_var"), tf.device("/cpu:0"):
            char_emb_mat = tf.get_variable("char_emb_mat", shape=[VC, dc], dtype='float')
    
        with tf.variable_scope("char"):
            Acx = tf.nn.embedding_lookup(char_emb_mat, self.cx)  # [N, M, JX, W, dc]
            Acq = tf.nn.embedding_lookup(char_emb_mat, self.cq)  # [N, JQ, W, dc]
            Acx = tf.reshape(Acx, [-1, JX, W, dc])
            Acq = tf.reshape(Acq, [-1, JQ, W, dc])
            
            filter_sizes = list(map(int, config.out_channel_dims.split(',')))
            heights = list(map(int, config.filter_heights.split(',')))
            assert sum(filter_sizes) == dco, (filter_sizes, dco)
            
            with tf.variable_scope("conv"):
                

In [129]:
config.out_channel_dims

'100'

In [130]:
config.filter_heights

'5'